# Exercise 4.17

LDA/QDA on height/weight data

In [1]:
from math import pi

import numpy as np
from numpy.linalg import det
from numpy.linalg import inv
import pandas as pd

## Load data

Load data and parse it into the design matrix X and the target values y

In [2]:
df = pd.read_table(
    '../../pmtk3/data/heightWeight/heightWeightData.txt', #Change this to the path to your heightWeightData.txt
    header=None, 
    delimiter=',')

In [3]:
columns = ['Sex', 'Height', 'Weight']
df.columns = columns
df.head()

,Sex,Height,Weight
0,1,67,125
1,2,68,140
2,2,67,142
3,2,60,110
4,2,64,97


In [4]:
dataset = df.values
X = dataset[:, 1:3]
y = dataset[:, 0]

## Remarks
I think Height and Weight are inverted, but I will follow the labels of heightWeightDataRaw.txt

## Classifiers

The fitting of the model will be performed using MLE, according with section 4.2.4
We will build classes for each classifier with the method fit, inspired in scikit-learn interface.

### QDA

In [9]:
class QDA:
        
    def __init__(self):
        
        self.mu = []
        self.Sigma = []
        self.prior = []
        self.classes = []
        
    def fit(self, X, y):
        '''Find mu, Sigma and the prior based on the MLE'''
        
        num_instances = X.shape[0]
        num_features = X.shape[1]        
        classes = set(y)
        
        for idx, class_ in enumerate(classes):            
            
            targets_idx = (y == class_)
            nc = len(y[targets_idx])                        
            prior = nc/num_instances
            
            mu = np.sum(X[targets_idx], axis=0)
            mu = mu/nc
            
            Sigma = np.zeros((num_features, num_features))
            X_centered = X - mu
            X_centered = X_centered[targets_idx]
            for idx in range(nc):
                Sigma += np.outer(X_centered[idx], X_centered[idx])
            Sigma = Sigma/nc                                
            
            self.prior.append(prior)
            self.mu.append(mu)
            self.Sigma.append(Sigma)
            self.classes.append(class_)
    
    def score(self, X, y):
        '''Calculate the accuracy of the model on the dataset'''
        
        num_instances = X.shape[0]
        predictions, _ = self.predict(X)
        acc = (y == predictions).sum()
        acc /= num_instances
        return acc
        
    
    def evaluate(self, x):
        '''Evaluate probability of each class'''
        
        evaluations = []
        normalization = 0
        for class_idx, class_ in enumerate(self.classes):                        
            mu = self.mu[class_idx]
            Sigma = self.Sigma[class_idx]
            prior = self.prior[class_idx]            
            quadratic = -0.5*np.dot((x - mu).T, inv(Sigma))
            quadratic = np.dot(quadratic, (x-mu))
            p = prior
            p *= det(2*pi*Sigma)**(-0.5)
            p *= np.exp(quadratic)
            normalization += p
            evaluations.append([class_, p])
        evaluations = ((x[0], x[1]/normalization) for x in evaluations)
        return evaluations    
    
    def predict(self, X):
        """Predict classes of instances X"""
            
        num_instances = X.shape[0]
        predictions = []
        probabilites = []
        for idx in range(num_instances):
            evaluation = self.evaluate(X[idx])
            prediction, prob = max(evaluation,key=lambda x:x[1]) # MAP decision criteria
            predictions.append(prediction)
            probabilites.append(prob)
        return predictions, probabilites

### LDA

The shared covariance matrix is calculated by summing all the outer product terms, with the right mean vector, and divide the final matrix by the total amount of instances.

In [12]:
class LDA:
        
    def __init__(self):
        
        self.mu = []
        self.Sigma = None
        self.prior = []
        self.classes = []
        
    def fit(self, X, y):
        '''Find mu, Sigma and the prior based on the MLE'''
        
        num_instances = X.shape[0]
        num_features = X.shape[1]                
        classes = set(y)
        self.Sigma = np.zeros((num_features, num_features))
                
        for idx, class_ in enumerate(classes):            
            
            targets_idx = (y == class_)
            nc = len(y[targets_idx])                    
            prior = nc/num_instances
            
            mu = np.sum(X[targets_idx], axis=0)
            mu = mu/nc
            
            Sigma = np.zeros((num_features, num_features))
            X_centered = X - mu
            X_centered = X_centered[targets_idx]
            for idx in range(nc):
                Sigma += np.outer(X_centered[idx], X_centered[idx])                                            
            self.prior.append(prior)
            self.mu.append(mu)                        
            self.Sigma += Sigma 
            self.classes.append(class_)
        self.Sigma = self.Sigma/num_instances
    
    def score(self, X, y):
        '''Calculate the accuracy of the model on the dataset'''
        
        num_instances = X.shape[0]
        predictions, _ = self.predict(X)
        acc = (y == predictions).sum()
        acc /= num_instances
        return acc
        
    
    def evaluate(self, x):
        '''Determine instance class'''
        
        evaluations = []
        normalization = 0
        for class_idx, class_ in enumerate(self.classes):                        
            mu = self.mu[class_idx]
            Sigma = self.Sigma
            prior = self.prior[class_idx]            
            gamma = -0.5*np.dot(mu.T, inv(Sigma))
            gamma = np.dot(gamma, mu)
            gamma += np.log(pi)
            beta = np.dot(inv(Sigma),mu)
            p = np.exp(np.dot(beta.T, x) + gamma)
            normalization += p                
            evaluations.append([class_, p])        
        evaluations = ((x[0], x[1]/normalization) for x in evaluations)
        return evaluations    
    
    def predict(self, X):
        """Predict classes of instances X"""
            
        num_instances = X.shape[0]
        predictions = []
        probabilites = []

        for idx in range(num_instances):
            evaluation = self.evaluate(X[idx])
            prediction, prob = max(evaluation,key=lambda x:x[1])
            predictions.append(prediction)
            probabilites.append(prob)
        return predictions, probabilites

## Determine misclassification

In [18]:
qda = QDA()
qda.fit(X, y)
score = qda.score(X, y)
misclassification = 1 - score
print('QDA misclassification rate on training set: {0:.3f}'.format(misclassification))

QDA misclassification rate on training set: 0.119


In [19]:
lda = LDA()
lda.fit(X, y)
score = lda.score(X, y)
misclassification = 1 - score
print('LDA .isclassification rate on training set: {0:.3f}'.format(misclassification))

LDA .isclassification rate on training set: 0.124


# Sanity check

In [30]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as sk_LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as sk_QDA

In [31]:
sk_qda = sk_QDA(priors=qda.prior)
sk_qda.fit(X, y)
score = sk_qda.score(X,y)
misclassification = 1 - score
print('QDA misclassification rate on training set: {0:.3f}'.format(misclassification))

QDA misclassification rate on training set: 0.119


In [33]:
sk_lda = sk_LDA(priors=lda.prior)
sk_lda.fit(X, y)
score = sk_lda.score(X,y)
misclassification = 1 - score
print('LDA .isclassification rate on training set: {0:.3f}'.format(misclassification))

LDA .isclassification rate on training set: 0.119


# Conclusion

The results show a reasonable result, where all the misclassifications rates were lower than 13 %. In my implementation, the LDA result was a little bit worse than the QDA (about 0.5 %). Making a sanity check with the scikit-learn implementation we observe an equal score for the QDA and a better score for the scikit LDA. The most likely cause is that they must calculate the shared covariance matrix in a different way (probably in a better way because scikit-learn is a pro machine learning library).